# Sequence evolution walkthrough
**Authorship:**
Adam Klie (last updated: *07/16/2023*)
***
**Description:**
This notebook is meant to serve as a guide for performing an *in silico* evolution analysis with a trained PyTorch model using SeqExplainer. The notebook will walk through the steps of performing the analysis and interpreting the results.
***

# Background
Generating functional DNA sequences is a huge research area in synthetic biology. From controlling plant behavior to , the possibilities are huge. Machine learning models offer signfiicant promise as a tool for generating functional sequences, but this area is still new. With SeqExplainer, we aim to give users tools for exploring the sequence space around a trained model. We implement the simplest form of this approach that iteratively evolves a sequence by greedily inserting the mutation with the largest predicted impact at each iteration. Starting with an initial sequence (e.g. random, shuffled, etc.), this strategy can be used to evolve synthetic functional sequences. This style of analysis is a promising direction for further research, and can also serve as an extension of ISM for validating that the model has learned representations that resemble motifs.

# Running an *in silico* evolution
Let's run an evolution on the model trained in the EUGENe paper for prediction of the regulatory activity of plant promoters.

We recommend cloning the entire tutorials repository so that you have all the necessary intermediate files you need, but when applicable, we also provide links to download the files directly.

In [ ]:
# Change this to where you would like to save all your results, likely your tutorials download directory
import os
os.chdir("/cellar/users/aklie/projects/ML4GLand/tutorials")  # TODO: change this to your own directory
cwd = os.getcwd()
cwd

In [ ]:
# TODO: Uncomment and run the following to get the model and config. You will not need to do this if you are working in your cloned tutorials repo
#!mkdir -p $cwd/models
#!wget https://github.com/ML4GLand/tutorials/raw/main/models/jores21_cnn_leaf.ckpt -O $cwd/models/jores21_cnn_leaf.ckpt

#!mkdir -p $cwd/configs
#!wget https://raw.githubusercontent.com/ML4GLand/tutorials/main/configs/cnn.yaml -O $cwd/configs/cnn.yaml

## Load in a model
 - TODO: Make the model checkpoint available somewhere

In [ ]:
# General purpose imports
import os
import torch
import numpy as np

In [ ]:
from eugene import models

In [ ]:
# Load up the model form the checkpoint
model_file = "models/jores21_cnn_leaf.ckpt"
model_arch = models.load_config(config_path="configs/cnn.yaml")
model = models.SequenceModule.load_from_checkpoint(model_file, arch=model_arch.arch)
model

## Generate some random sequences
We next need to create the sequences we want to evolve. In this notebook, we will generate random sequences as our starting material. We can use SeqPro functions to do this.

In [ ]:
import seqpro as sp

In [ ]:
# Generate 10 sequences of length 170
random_seqs = sp.random_seqs(
    shape=(10, 170),
    alphabet=sp.alphabets.DNA
)

In [ ]:
# One-hot encode the sequences
random_seqs = sp.ohe(random_seqs, alphabet=sp.alphabets.DNA).transpose(0, 2, 1)

## Perform the evolution
Now that we have some baseline sequences, we can perform the evolution. This is done by iteratively performing a *in silico* saturation mutagenesis and selecting the mutation that most increases the predicted activity.

In [ ]:
from seqexplainer import evolution
from tqdm.auto import tqdm  # progress bar to use for evolution

In [ ]:
# Takes about 2 minutes to do 5 rounds of evolution on 10 sequences
rounds = 5
evolved_seqs = []
deltas = np.zeros((len(random_seqs), rounds))
mutation_pos = []
for i, random_seq in tqdm(enumerate(random_seqs), total=random_seqs.shape[0], desc="Evolving sequences"):
    evolved_res = evolution(
        model=model,
        X=random_seq,
        rounds=rounds,
    )
    evolved_seqs.append(evolved_res[0])
    deltas[i, :] = deltas[i, :] + evolved_res[1]
    mutation_pos.append(evolved_res[2])
X_evolved = torch.tensor(np.array(evolved_seqs), dtype=torch.float32)
mutation_pos = np.array(mutation_pos)

## Plot the score change over rounds
Let's visualize the results by plotting the violin plots predicted activities over each round of evolution. The easiest way to do this is to first create a dataframe with the results.

In [ ]:
import pandas as pd

In [ ]:
scores = {}
scores[0] = model.predict(random_seqs, batch_size=32, verbose=False).detach().cpu().numpy().squeeze()
for i in range(1, rounds+1):
    scores[i] = scores[i-1] + deltas[:, i-1]
score_df = pd.DataFrame(scores)
score_df.columns = ["round_{}".format(i) for i in range(rounds+1)]
score_df.head()

Now we can easily plot with seaborn

In [ ]:
import seaborn as sns

In [ ]:
# Make a violinplot of the scores at each round
g = sns.violinplot(data=score_df, color="lightblue")
g.set_xlabel("Round")
g.set_ylabel("Score")

## Plot the differences in attribution logos
Another interesting visualization one can use is to plot the before and after attrribution logos. This allows us to see what features are being selected for by the model.

In [ ]:
from seqexplainer import attribute

In [ ]:
# Get the attributions for the random and evolved sequences
random_attrs = attribute(
    model=model,
    inputs=random_seqs,
    method="InputXGradient",
)
evolved_attrs = attribute(
    model=model,
    inputs=X_evolved,
    method="InputXGradient",
)

We can plot the attribution logos using logomaker

In [ ]:
import matplotlib.pyplot as plt
import logomaker as lm

In [ ]:
# Complicated plotting that we will eventually turn into a built in function
for i in tqdm(range(len(random_attrs)), desc="Plotting", total=len(random_attrs)):
    fig, ax = plt.subplots(2, 1, figsize=(10, 4))
    random_viz_seq = pd.DataFrame(random_attrs[i].T, columns=["A", "C", "G", "T"])
    random_viz_seq.index.name = "pos"
    random_logo = lm.Logo(random_viz_seq, color_scheme="classic", figsize=(10, 2), ax=ax[0])
    random_logo.style_spines(visible=False)
    random_logo.style_spines(spines=['left'], visible=True)
    random_logo.ax.set_xticks([])
    ax[0].set_title(f"{scores[0][i].item():.2f} -> {scores[5][i].item():.2f}")
    ax_bottom = ax[0].get_ylim()[0]
    evolved_viz_seq = pd.DataFrame(evolved_attrs[i].T, columns=["A", "C", "G", "T"])
    evolved_viz_seq.index.name = "pos"
    evolved_logo = lm.Logo(evolved_viz_seq, color_scheme="classic", figsize=(10, 2), ax=ax[1])
    evolved_logo.style_spines(visible=False)
    evolved_logo.style_spines(spines=['left'], visible=True)
    evolved_logo.ax.set_xticks([])

Unsurprisingly for this model, we see that it tries to select for the presence of the TATA box in almost every case (even when one already exists). And it doesn't even seem to care if it puts them only a few nucleotides apart.

## Avoiding overinterpretation
Sequence evolution represents abother means for intepreting what features a model is learning to use to make predictions and represents an exxciting avenue for exploration. As a newer method, one should also be aware that theoretical properties of prediction evolution are not well understood and should be investigated. We hope SeqExplainer provides a useful tool for exploring this area.

# DONE!

---

# Scratch